In [1]:
# 라이브러리 선언
import sqlite3
import csv
import re
import os

CSV 파일을 SQLite 데이터베이스로 전환하는 함수를 선언합니다.

In [2]:

# CSV 파일을 SQLite 데이터베이스로 전환하는 함수 선언
def csv_to_sqlite(csv_filepath, db_filepath, table_name, drop_existing_table=False):
    """
    CSV 파일을 읽어 SQLite 데이터베이스로 전환합니다.

    Args:
        csv_filepath (str): 변환할 CSV 파일의 경로.
        db_filepath (str): 생성하거나 연결할 SQLite 데이터베이스 파일의 경로.
        table_name (str): 데이터를 삽입할 SQLite 테이블의 이름.
    """
    try:
        conn = sqlite3.connect(db_filepath)
        cursor = conn.cursor()

        # drop_existing_table이 True이면 기존 테이블 삭제
        if drop_existing_table:
            drop_table_sql = f"DROP TABLE IF EXISTS {table_name};"
            cursor.execute(drop_table_sql)
            print(f"기존 테이블 '{table_name}'이(가) 삭제되었습니다 (존재하는 경우).")
            conn.commit() # DROP TABLE 후에는 바로 커밋하여 반영

        with open(csv_filepath, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            headers = next(reader)  # 첫 번째 행을 헤더로 읽음

            # SQL 컬럼 이름으로 유효하게 변환 (특수 문자 제거 및 공백 대체)
            cleaned_headers = [re.sub(r'\W+', '', header).strip().lower() for header in headers]
            
            # 테이블 생성 쿼리 구성
            # 모든 컬럼을 기본적으로 TEXT 타입으로 지정하여 유연성을 높입니다.
            # 필요에 따라 INSERT 전에 데이터 타입 변환 로직을 추가할 수 있습니다.
            columns_def = ', '.join([f"{header} TEXT" for header in cleaned_headers])
            create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_def});"
            cursor.execute(create_table_sql)
            print(f"테이블 '{table_name}'이(가) 생성되었거나 이미 존재합니다.")

            # 데이터 삽입 쿼리 구성
            placeholders = ', '.join(['?' for _ in cleaned_headers])
            insert_sql = f"INSERT INTO {table_name} ({', '.join(cleaned_headers)}) VALUES ({placeholders});"

            # 데이터 삽입
            for row_num, row in enumerate(reader, start=2): # 첫 번째 행은 헤더이므로 2부터 시작
                if len(row) != len(cleaned_headers):
                    print(f"경고: {row_num}행의 컬럼 개수가 헤더와 일치하지 않습니다. 이 행은 건너뜁니다: {row}")
                    continue
                try:
                    cursor.execute(insert_sql, row)
                except sqlite3.Error as e:
                    print(f"오류: {row_num}행을 삽입하는 중 오류가 발생했습니다: {row}. 오류 메시지: {e}")
                    continue # 오류 발생 시 해당 행은 건너뛰고 다음 행으로 진행

            conn.commit()
        print(f"'{csv_filepath}' 파일의 데이터가 '{db_filepath}'의 '{table_name}' 테이블로 성공적으로 전환되었습니다.")

    except FileNotFoundError:
        print(f"오류: '{csv_filepath}' 파일을 찾을 수 없습니다. 경로를 확인해 주세요.")
    except sqlite3.Error as e:
        print(f"SQLite 데이터베이스 오류: {e}")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")
    finally:
        if 'conn' in locals() and conn:
            conn.close()


CSV 파일을 SQLite 데이터베이스로 전환합니다.  
기존에 테이블이 존재한다면 삭제 후 재생성합니다.  
만약 아래와 같은 오류가 발생한다면 CSV 파일에 DRM 이 걸려있는지 확인해주세요.  
```
예상치 못한 오류가 발생했습니다: 'utf-8' codec can't decode byte 0x9b in position 0: invalid start byte
```

In [3]:
# CSV 파일을 SQLite 데이터베이스로 전환하는 함수 호출
csv_file = "../data/임베딩대상_202505261852.csv"
sqlite_db = "../.db/sqlite/my_database.db"
table = "PRODUCT"


# SQLite DB 폴더 존재 여부 확인 및 생성
db_dir = os.path.dirname(sqlite_db)
if not os.path.exists(db_dir):
    os.makedirs(db_dir)


csv_to_sqlite(csv_file, sqlite_db, table, True)

# 3. 데이터 확인 (선택 사항)
print("\n--- SQLite 데이터 확인 ---")
try:
    conn = sqlite3.connect(sqlite_db)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table};")
    rows = cursor.fetchall()
except sqlite3.Error as e:
    print(f"데이터 확인 중 오류 발생: {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()

# 샘플 출력
for row in rows[:10]:
    print(row)

기존 테이블 'PRODUCT'이(가) 삭제되었습니다 (존재하는 경우).
테이블 'PRODUCT'이(가) 생성되었거나 이미 존재합니다.
'../data/임베딩대상_202505261852.csv' 파일의 데이터가 '../.db/sqlite/my_database.db'의 'PRODUCT' 테이블로 성공적으로 전환되었습니다.

--- SQLite 데이터 확인 ---
('0021216860', '엔산 EL-09 이젤형 TV스탠드 티비거치대', '정상상품', '엔산마운트', 'A/V 기타>TV 기타', '색상', '화이트 계열', 'TV·영상가전>TV>TV액세서리', 'N', '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', '176000', '176000', '0', '', '', '176000')
('0026595956', '엔산 무드조명형 모니터암 블룸스탠드 삼텐바이미 거치대', '정상상품', '엔산마운트', 'A/V 기타>TV 기타', '', '', 'TV·영상가전>TV>TV액세서리', 'N', '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', '140000', '140000', '0', '', '', '140000')
('0037357623', '5년무상AS 이스트라 120Hz AI 맥스 127cm UHD 구글 5.0 스마트TV (이젤스탠드A타입-기사)', '정상상품', '이스트라', 'TV>UHD TV', '크기,종류,해상도,응답속도,기본 주사율,스마트기능,스마트기능,스마트기능,스마트기능,스마트기능,스마트기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,e효율등급,출시년도,화면 타입,스피커 출력,음향효과,음향효과,지원단자', '50인치(127cm),LED,UHD(4K),6m/s,120Hz,구글플레이,넷플릭스,유튜브,크롬캐스트,구글어시스턴트,Disney+,블루투스,게임모드,플리커프리,HDMI CEC,HDMI 2.1 지원,HDMI eARC,크로마서브샘플링(4:4:4),USB재생(동영상·사진·음악),1등급,2025년,평면형,24W,